# Databases

In [3]:
import sqlite3

## Create DB

- Create a new sqlite3 database called `library`.

In [2]:
conn = sqlite3.connect('library.db')
cursor = conn.cursor()

## Create Table

Create a table named `books` with the following characteristics:
- id INTEGER PRIMARY KEY
- title TEXT
- author TEXT
- published_date DATE

In [17]:
cursor.execute('''
                   CREATE TABLE library
                   (id INTEGER PRIMARY KEY,
                   title TEXT,
                   author TEXT,
                   published_date DATE)
                ''')

conn.commit()

OperationalError: table library already exists

## Add Book

Write a Python function `add_book()` that will add a new book to the "books" table.

In [26]:
def add_book(conn, cursor, title, author, published_date):
    cursor.execute('''INSERT INTO library (title, author, published_date)
                  VALUES (?, ?, ?)''', (title, author, published_date,))
    conn.commit()

In [27]:
add_book(conn, cursor, 'The Lord of the Rings', 'J.R.R. Tolkien', '1954-07-29')
add_book(conn, cursor, 'Harry Potter', 'J.K. Rowling', '1997-06-26')

## Get Book

Write a Python function `get_books_by_author()` that will retrieve all books by a given author.

In [38]:
def get_books_by_author(conn, cursor, author):
    cursor.execute("""
                    SELECT * FROM library
                   where author = ?
                    """, (author,))
    print(cursor.fetchall())

In [29]:
get_books_by_author(conn, cursor, 'J.R.R. Tolkien')

[(1, 'The Lord of the Rings', 'J.R.R. Tolkien', '1954-07-29')]


In [25]:
get_books_by_author(conn, cursor, 'J.R.R. Tolkien')

[(1,
  'Lord of the Rings: The Fellowship of the Ring',
  'J.R.R. Tolkien',
  '1954-07-29')]

## Update Book

Write a Python function `update_book_title()` that will update the title of a book given its id.

In [30]:
def update_book_title(conn, cursor, id, title):
    cursor.execute("""
                    UPDATE library set  title = ?
                   where id = ?""", (title, id))
    conn.commit()

In [31]:
update_book_title(conn, cursor, 1, 'Lord of the Rings: The Fellowship of the Ring')
get_books_by_author(conn, cursor, 'J.R.R. Tolkien')

[(1, 'Lord of the Rings: The Fellowship of the Ring', 'J.R.R. Tolkien', '1954-07-29')]


In [27]:
update_book_title(conn, cursor, 1, 'Lord of the Rings: The Fellowship of the Ring')

In [28]:
get_books_by_author(conn, cursor, 'J.R.R. Tolkien')

[(1,
  'Lord of the Rings: The Fellowship of the Ring',
  'J.R.R. Tolkien',
  '1954-07-29')]

## Delete Book

Write a Python function `delete_book()` that will delete a book from the "books" table given its id.

In [32]:
def delete_book(conn, cursor, id):
    cursor.execute("""DELETE from library where id = ?""", (id,))
    conn.commit()

In [33]:
delete_book(conn, cursor, 2)
get_books_by_author(conn, cursor, 'J.K. Rowling')

[]


In [30]:
delete_book(conn, cursor, 2)

In [31]:
get_books_by_author(conn, cursor, 'J.K. Rowling')

[]

## Published Before

Write a Python function `get_books_published_before()` that will retrieve all books published before a given date.

In [40]:
def get_books_published_before(conn, cursor, date):
    cursor.execute("""Select * from library where published_date < ?""", (date,))
    print(cursor.fetchall())

In [41]:
get_books_published_before(conn, cursor, '2000-01-01')

[(1, 'Lord of the Rings: The Fellowship of the Ring', 'J.R.R. Tolkien', '1954-07-29')]


In [33]:
get_books_published_before(conn, cursor, '2000-01-01')

[(1,
  'Lord of the Rings: The Fellowship of the Ring',
  'J.R.R. Tolkien',
  '1954-07-29')]

## Closing

Close the cursor and the connection.

In [42]:
cursor.close()
conn.close()

## Guess the Movie

Create a game where the user is presented with characters from a movie title replaced by underscores and must guess the movie title. Users have limited attempts, and their scores are recorded in a database.

### Create DB

Create a new database called "guess_movie.db".

In [43]:
conn = sqlite3.connect('guess_movie.db')
cursor = conn.cursor()

### Tables Setup

Create a table called "movies" with columns "id" and "title" and another table "scores" with columns "id," "username," and "score."

In [46]:
cursor.execute('''
                   CREATE TABLE movies
                   (id INTEGER PRIMARY KEY,
                   title TEXT)
                ''')
cursor.execute('''
                CREATE TABLE scores
                (id INTEGER PRIMARY KEY,
                    username TEXT,
                    score INTEGER)
                ''')

conn.commit()

### Populate DB

Initialize the DB with the following movies:

In [10]:
movie_titles = [
    "The Shawshank Redemption",
    "The Godfather",
    "The Dark Knight",
    "Pulp Fiction",
    "Forrest Gump",
    "Inception",
    "Fight Club",
    "The Matrix",
    "The Social Network",
    "Jurassic Park",
    "Titanic",
    "Toy Story",
    "Finding Nemo",
    "The Lord of the Rings: The Return of the King",
    "Star Wars: Episode IV - A New Hope",
    "The Silence of the Lambs",
    "Se7en",
    "The Lion King",
    "Back to the Future",
    "The Usual Suspects"
]

In [11]:
cursor.executemany("""
                    INSERT INTO movies (title) VALUES (?)
                    """, [(title,) for title in movie_titles])

conn.commit

NameError: name 'cursor' is not defined

### The Game

- You should create a Tkinter window with a label to display the masked movie title, an entry widget to input the guessed movie title, a button to submit the guess, and a label to display the remaining attempts.
- When the game starts, pick a random movie title from the database, mask it with underscores, and display it to the user. Each time the user submits a guess, check it against the actual movie title. If the guess is correct, the user wins, and their score is updated in the database.

A good way to organize the code may be:
- Create a `MovieDatabase` class that encapsulates all functionalities related to database management, including inserting movie titles, fetching a random movie, and closing the database connection.
- Create a `GuessTheMovieGame` class that handles all the game logic and GUI functionalities, and it utilizes an instance of `MovieDatabase` to interact with the database.
- Define a `main()` function that runs the game, and ensures that the database connection is closed appropriately upon application termination.

This is just a proposal, feel free to organize your program as you wish.

In [9]:
import tkinter as tk
import random


class MovieDatabase:
    def __init__(self):
        self.conn = sqlite3.connect('guess_movie.db')
        self.cursor = self.conn.cursor()

    def insert_movie(self, title):
        self.cursor.execute("""
            INSERT INTO movies (title) VALUES (?)
        """, (title,))
        self.connection.commit()
        self.close_connection

    def fetch_random_movie(self):
        self.cursor.execute("SELECT title FROM movies")
        print(self.cursor.fetchall())
        movies = list(self.cursor.fetchall())
        print(movies)
        random_movie = random.choice(movies)[0]
        self.close_connection()
        return random_movie

    def close_connection(self):
        self.cursor.close()
        self.conn.close()

    def insert_player(self, name, score):
        self.cursor.execute("INSERT INTO scores (username, score) VALUES (?, ?)", (name, score,))
        self.connection.commit()
        self.close_connection()

class GuessTheMovieGame():
    def __init__(self, movie):
        self.movie = movie
        self.current_movie = ""
        self.intents = 0
        self.game()
        
    def hide_movie_title(self):
        hidden_title = ""
        for i in self.current_movie:
            if i != " ":
                hidden_title += "*"
            else:
                hidden_title += " "
        return hidden_title
    
    def game(self):
        self.current_movie = self.movie.fetch_random_movie()
        root = tk.Tk()
        root.title("The GAME")

        label4 = tk.Label(root, text="Username")
        label4.pack(side=tk.LEFT)

        entry2 = tk.Entry(root)
        entry2.pack(side=tk.LEFT)

        label = tk.Label(root, text="Title Movie")
        label.pack(side=tk.LEFT)
        label2 = tk.Label(root, text="Title Movie ***")
        label2.pack(side=tk.LEFT)

        entry = tk.Entry(root)
        entry.pack(side=tk.LEFT)

        label3 = tk.Label(root, text="Tries")
        label3.pack(side=tk.LEFT)

        button = tk.Button(root, text="Try", command=self.on_button_click_try(entry, label3, entry2))
        button.pack(side=tk.LEFT)   
        button2 = tk.Button(root, text="Insert Mov", command=self.insert_mov(entry, label3))
        button2.pack(side=tk.LEFT)   

        root.mainloop()

    def on_button_click_try(self,entry, label3, entry2):
        title = entry.get()
        username = entry2.get()
        cont = 0
        if self.current_movie == title:
            label3.config(text = "Congratulations you win!")
            self.movie.insert_player(username, cont)
        else:
            cont += 1
            label3.config(text = cont)

    def insert_mov(self, entry, label3):
        title = entry.get()
        self.movie.insert_movie(title)
        label3.config(text="Title added")
        


def main():
    mov = MovieDatabase()
    gam = GuessTheMovieGame(mov)
    gam.game()

if __name__ == "__main__":
    main()


[]
[]


IndexError: Cannot choose from an empty sequence

### Improvements

- Create a "High Scores" section that lists the top 10 scores from the "scores" table.
- Include a hint feature, revealing a character of the movie title after a certain number of incorrect guesses.
- Add an admin section where you can add/delete movies to/from the database.
- Add multiplayer functionality.